In [15]:
import pandas as pd
import numpy as np
from scipy.stats import norm, gaussian_kde


In [16]:
df = pd.read_csv("Natixis Stock.csv", header=None, sep=r"\s+", names=["Date", "Price"])
df.head()

,Date,Price
0,02/01/2015,"5,621"
1,05/01/2015,"5,424"
2,06/01/2015,"5,329"
3,07/01/2015,"5,224"
4,08/01/2015,"5,453"


In [17]:
df["Date"]=pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
df["Price"]=df["Price"].str.replace(",",".").astype(float)

In [18]:
print(df.head(5))
print(df.tail(5))
print(df.describe())
print("Dates:", df.index.min(), "→", df.index.max())
print("Nb lignes:", len(df), "| Nb NaN price:", df["Price"].isna().sum())
print("Prix > 0 ? ", (df["Price"] > 0).all())

        Date  Price
0 2015-01-02  5.621
1 2015-01-05  5.424
2 2015-01-06  5.329
3 2015-01-07  5.224
4 2015-01-08  5.453
           Date  Price
1018 2018-12-21  4.045
1019 2018-12-24  4.010
1020 2018-12-27  3.938
1021 2018-12-28  4.088
1022 2018-12-31  4.119
                                Date        Price
count                           1023  1023.000000
mean   2016-12-30 10:54:32.727272704     5.684600
min              2015-01-02 00:00:00     3.077000
25%              2016-01-02 00:00:00     4.927000
50%              2016-12-29 00:00:00     5.782000
75%              2017-12-28 12:00:00     6.532000
max              2018-12-31 00:00:00     7.744000
std                              NaN     1.021034
Dates: 0 → 1022
Nb lignes: 1023 | Nb NaN price: 0
Prix > 0 ?  True


In [19]:
# la date va que jusqu'à decembre 2016
df1 =df[(df["Date"]>="2015-01-01") & (df["Date"]<="2016-12-31")]
df1 = df1.iloc[1:,:]
df = df.iloc[1:,:]
print(df.tail(5))

           Date  Price
1018 2018-12-21  4.045
1019 2018-12-24  4.010
1020 2018-12-27  3.938
1021 2018-12-28  4.088
1022 2018-12-31  4.119


In [20]:
alpha=0.05

In [21]:
df1["return"]=df1["Price"].pct_change()
df["return"]=df["Price"].pct_change()

varEmp=df1["return"].quantile(alpha)
print(f"Empirical VaR at {(1-alpha) *100:.2f}% : {varEmp*100}%") #il faut que ce soit négatif

Empirical VaR at 95.00% : -3.788407989779058%


In [22]:
mu = df1['return'].mean()
sigma = df1['return'].std()
z = norm.ppf(alpha)
varParam = (mu + sigma * z)# Generate 100'000 standard normal random values
print(f"Parametric VaR ({(1 - alpha) * 100:.2f}%): {varParam*100:.6f}%")

Parametric VaR (95.00%): -3.898272%


In [23]:
insample   = df1[(df1["Date"] <= "2016-12-31")]["return"].dropna()
outsample  = df[(df["Date"] >= "2017-01-01") & (df1["Date"] <= "2018-12-31")]["return"].dropna()

var_hist = -np.nanpercentile(insample, alpha*100)

exc_in  = np.mean(insample < -var_hist)
exc_out = np.mean(outsample < -var_hist)

print(f"In-sample exceedances  = {exc_in*100:.2f}%")
print(f"Out-of-sample exceed. = {exc_out*100:.2f}%")#problème ici car comme on prend à l'origine 2015 --> 2016 et que mtn on veux prendre le reste, on a déjà nettoyer les données

In-sample exceedances  = 5.09%
Out-of-sample exceed. = nan%


In [24]:
#var paramétrique --> peut avoir des erreurs de paramètres
#on utilise donc la var non param pour eviter ces erreurs de paramètre 

In [25]:
#calcul de la VaR avec moins de points sur deux mois uniquement
#  Question 3
df2 =df[(df["Date"]>="2016-10-01") & (df["Date"]<="2016-12-31")]
varEmp=df2["return"].quantile(alpha)
print(f"Empirical VaR at {(1-alpha) *100:.2f}% : {varEmp*100}%") #il faut que ce soit négatif

Empirical VaR at 95.00% : -2.4483182167546773%


In [ ]:
mu = df2['return'].mean()   
sigma = df2['return'].std()
z = norm.ppf(alpha)
varParam = (mu + sigma * z)# Generate 100'000 standard normal random values
print(f"Parametric VaR ({(1 - alpha) * 100:.2f}%): {varParam*100:.6f}%")

Parametric VaR (95.00%): -2.342231%


# TD2


In [36]:
df = df.dropna(subset=['return']).reset_index(drop=True)
df.head()

,Date,Price,return
0,2015-01-06,5.329,-0.017515
1,2015-01-07,5.224,-0.019704
2,2015-01-08,5.453,0.043836
3,2015-01-09,5.340,-0.020723
4,2015-01-12,5.264,-0.014232


In [ ]:
S_T = df['Price'].iloc[-1]
T_last = df['Date'].iloc[-1]

print(f"Date de calcul T: {T_last.strftime('%Y-%m-%d')}")
print(f"Dernier prix de l'action Natixis S_T: {S_T:.4f}")
print("-" * 50)

lambda_ewma = 0.94
returns = df['return'].values
T_data = len(returns)

ewma_variance = np.zeros(T_data)
ewma_variance[0] = returns[0]**2 # Initialisation de la variance avec le carré du premier rendement

for t in range(1, T_data):
    ewma_variance[t] = (1 - lambda_ewma) * returns[t-1]**2 + lambda_ewma * ewma_variance[t-1]

# Volatilité quotidienne et annualisée
sigma_daily = np.sqrt(ewma_variance[-1])
mu_daily = 0.0 # Drift négligé pour un horizon d'un jour

# Annualisation pour Black-Scholes (252 jours de trading)
sigma_ann = sigma_daily * np.sqrt(252)

print(f"Volatilité quotidienne EWMA (sigma_daily): {sigma_daily:.6f}")
print(f"Volatilité annualisée (sigma_ann): {sigma_ann:.6f}")
print("-" * 50)


def black_scholes_call(S, K, T, r, q, sigma):
    # Si volatilité est nulle
    if np.any(sigma <= 1e-6) or np.any(T <= 1e-6):
        return np.maximum(S - K, 0)

    S, K, T, r, q, sigma = map(np.asanyarray, (S, K, T, r, q, sigma))
    d1 = (np.log(S / K) + (r - q + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Paramètres de l'option (Call at the money, Maturité 1 mois, r=0, q=0)
K = S_T         
T_mat = 1/12   
r = 0.0         
q = 0.0       
sigma_BS = sigma_ann

# Prix initial du Call
C_T = black_scholes_call(S_T, K, T_mat, r, q, sigma_BS)

print(f"Paramètres du Call: Strike K={K:.4f}, Maturité T={T_mat:.4f} années")
print(f"Prix initial du Call: {C_T:.6f}")
print("-" * 50)


N_simulations = 10000 
np.random.seed(42)

Z = np.random.normal(0, 1, N_simulations)

# Simulation des rendements logarithmiques (lr_T+1)
lr_T_plus_1 = mu_daily + sigma_daily * Z

# Simulation des prix de l'action au jour T+1 (S_T+1)
S_T_plus_1 = S_T * np.exp(lr_T_plus_1)

# Calcul des prix simulés du Call (C_T+1)
C_T_plus_1 = black_scholes_call(S_T_plus_1, K, T_mat, r, q, sigma_BS)

# Calcul de la variation arithmétique du prix du Call (Delta C)
delta_C = C_T_plus_1 - C_T

# --- 5. Calcul de la VaR 99% ---
alpha = 0.01 # Pour une VaR à 99%

# La VaR (mesure de perte) est l'opposé du quantile alpha des variations (Profit/Perte)
VaR_99 = -np.quantile(delta_C, alpha)

print(f"Nombre de simulations Monte-Carlo (N): {N_simulations}")
print(f"VaR 99% sur la variation arithmétique du prix du Call (horizon 1 jour): {VaR_99:.6f}")

# (Bonus: Calcul de la CVaR 99% / Expected Shortfall)
CVaR_99 = -np.mean(delta_C[delta_C < -VaR_99])
print(f"CVaR 99% (Expected Shortfall): {CVaR_99:.6f}")

Date de calcul T: 2018-12-31
Dernier prix de l'action Natixis S_T: 4.1190
--------------------------------------------------
Volatilité quotidienne EWMA (sigma_daily): 0.024930
Volatilité annualisée (sigma_ann): 0.395759
--------------------------------------------------
Paramètres du Call: Strike K=4.1190, Maturité T=0.0833 années
Prix initial du Call C_T: 0.187631
--------------------------------------------------
Nombre de simulations Monte-Carlo (N): 10000
VaR 99% sur la variation arithmétique du prix du Call (horizon 1 jour): 0.098178
CVaR 99% (Expected Shortfall): 0.109103
